In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as XGBRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
df = pd.read_pickle('df_model')

In [3]:
df = df.dropna(subset =['final_years', 'final_make'])
df =df.drop(['make','make_title', 'year', 'years', 'title_lower', 'URL', 'bikeTitle', 'attributes',], axis =1)
df = df[df['final_years'].astype(str).str.isdigit()]
df['odometer'] = df.odometer.str.extract('(^\d*)')
df[["final_years"]] = df[['final_years']].apply(pd.to_numeric)
df['Price']= df['Price'].apply(lambda x: x.replace('$',''))
df['Price']= df['Price'].apply(lambda x: x.replace(',',''))
df[['Price']] = df[['Price']].apply(pd.to_numeric)
df[['odometer']] = df[['odometer']].apply(pd.to_numeric)
df =df.reset_index(drop=True)
df = df[df['Price']<50000]
df=df[df['Price']>600]
df = df[df['odometer']<100000]
df = df[df['final_years']>1960]
df = df[df['final_years']<2022]
df.drop("VIN", inplace =True, axis =1)
df =df.reset_index(drop=True)
df['condition'] = df['condition'].str.replace('excellentcryptocurrency ok', 'excellent')
df['condition'] = df['condition'].str.replace('like newcryptocurrency ok', 'like new')
df['condition'] = df['condition'].str.replace('goodcryptocurrency ok', 'good')
df['condition'] = df['condition'].str.replace('faircryptocurrency ok', 'fair')
df['condition'] = df['condition'].str.replace('newcryptocurrency ok', 'new')

In [4]:
df['fuel'] = df['fuel'].fillna(value='other')
df['paint color'] = df['paint color'].fillna(value='other')
df['title status'] = df['title status'].fillna(value='other')
df['transmission'] = df['transmission'].fillna(value='other')
df['condition'] = df['condition'].fillna(value='other')
df['engine displacement (CC)'] = df['engine displacement (CC)'].fillna(value='other')
df['type'] = df['type'].fillna(value='other')
df = df[df['engine displacement (CC)']!='other']

In [5]:
# df['engine displacement (CC)']= df['engine displacement (CC)'].astype(float)
# bins = [0,250,500,750,1000,1250,1500,1750,2000, np.inf]
# names =['0-250', '251-500', '501-750', '751-1000', '1001- 1250', '1251-1500', '1501-1750',' 1751-2000', '2001+']

# df['engine_displacement'] = pd.cut(df['engine displacement (CC)'], bins, labels=names)

In [6]:
# df = df.drop('engine displacement (CC)',axis=1)

In [7]:
df

,Price,title,fuel,odometer,paint color,title status,transmission,condition,engine displacement (CC),type,final_make,final_years
3,3000,2002 Honda VTX 1800 C Custom,gas,69000.0,black,clean,manual,excellent,1800,cafe racer,honda,2002
10,5500,1975 Honda CB500T Cafe Racer,gas,368.0,other,other,manual,excellent,534,cafe racer,honda,1975
11,2900,2004 Honda CRF450R,gas,100.0,other,clean,manual,excellent,450,dirtbike,honda,2004
12,5995,2016 Honda NM4,gas,13000.0,black,clean,automatic,excellent,700,cruiser,honda,2016
18,6900,R1200 GS Adventure,gas,71900.0,red,clean,automatic,good,1200,touring,bmw,2007
...,...,...,...,...,...,...,...,...,...,...,...,...
12478,1900,Suzuki GZ250 Marauder,gas,2793.0,black,clean,manual,excellent,250,cruiser,suzuki,2007
12480,19995,'20 BMW R1250GSA,gas,11715.0,green,clean,manual,excellent,1250,other,bmw,2020
12481,3200,2004 V-Strom 1000,gas,42000.0,blue,clean,manual,fair,1000,street,suzuki,2004
12482,7000,2012 Kawasaki Vulcan 900 Classic,gas,17543.0,black,clean,manual,excellent,900,cruiser,kawasaki,2012


In [8]:
# R-squared and RMSE Function
def performance(y_true, y_predict):
    """ 
    Calculates and returns the two performance scores between 
    true and predicted values - first R-Squared, then RMSE
    """

    # Calculate the r2 score between 'y_true' and 'y_predict'
    r2 = r2_score(y_true, y_predict)

    # Calculate the root mean squared error between 'y_true' and 'y_predict'
    rmse = mean_squared_error(y_true, y_predict, squared=False)

    # Return the score
    return [r2, rmse]

In [9]:
# Use train_test_split to create training data and testing data
X = df.drop(['Price'], axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
# Get categoricals
categoricals = ['fuel','paint color','title status', 'transmission', 'condition', 'type', 'final_make','engine displacement (CC)']

train_dummies = X_train[categoricals]
test_dummies = X_test[categoricals]

# Create OneHotEncoder object to create dummies
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform the dataset into dummies matrix
enc = ohe.fit(train_dummies)

train_dummies_trans = enc.transform(train_dummies)
test_dummies_trans = enc.transform(test_dummies)

# Dummies values in matrix form
train_data = train_dummies_trans.todense()
test_data = test_dummies_trans.todense()

# New dummy column names
names = ohe.get_feature_names(categoricals)

# Make them into Dataframe
train_dummies_trans_df = pd.DataFrame(train_data, columns=names)
test_dummies_trans_df = pd.DataFrame(test_data, columns=names)

In [11]:
train_dummies_trans_df = train_dummies_trans_df[train_dummies_trans_df.columns.drop(list(train_dummies_trans_df.filter(regex='other')))]
test_dummies_trans_df = test_dummies_trans_df[test_dummies_trans_df.columns.drop(list(test_dummies_trans_df.filter(regex='other')))]

In [12]:
train_dummies_trans_df.reset_index(drop=True, inplace=True)


test_dummies_trans_df.reset_index(drop=True, inplace=True)

X_train =X_train.drop(categoricals, axis=1)
X_test = X_test.drop(categoricals, axis=1)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

X_train_comb = pd.concat([X_train, train_dummies_trans_df], axis=1)
X_test_comb = pd.concat([X_test, test_dummies_trans_df], axis=1)

In [13]:
X_train_comb = X_train_comb.drop(['title'],axis=1)
X_test_comb = X_test_comb.drop(['title'], axis=1)

In [ ]:
X_train_comb['engin']

In [122]:
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

In [123]:
model = XGBRegressor()

In [124]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [125]:
scores = cross_val_score(model, X_test_comb, y_test, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [126]:
scores = absolute(scores)

In [127]:
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 1803.262 (149.621)


In [128]:
# R-squared and RMSE Function
def performance(y_true, y_predict):
    """ 
    Calculates and returns the two performance scores between 
    true and predicted values - first R-Squared, then RMSE
    """

    # Calculate the r2 score between 'y_true' and 'y_predict'
    r2 = r2_score(y_true, y_predict)

    # Calculate the root mean squared error between 'y_true' and 'y_predict'
    rmse = mean_squared_error(y_true, y_predict, squared=False)

    # Return the score
    return [r2, rmse]

In [129]:
model.fit(X_train_comb, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [130]:
y_pred = model.predict(X_test_comb)
score = performance(y_test, y_pred)
score[0], score[1], model

(0.7590707866125307,
 2270.7291583330752,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [131]:
param_grid = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [6],
    'min_child_weight': [1, 2],
    'subsample': [0.5, 0.7],
    'n_estimators': [100],
}

In [132]:
grid_clf = GridSearchCV(model, param_grid, cv=3, n_jobs=3)
grid_clf.fit(X_train_comb, y_train)

best_parameters = grid_clf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train_comb)
test_preds = grid_clf.predict(X_test_comb)

score = performance(y_test, test_preds)
print(score[0], score[1], model)

/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Grid Search found the following optimal parameters: 
learning_rate: 0.2
max_depth: 6
min_child_weight: 2
n_estimators: 100
subsample: 0.7
0.7644243419275982 2245.3591224085644 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)


In [133]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [134]:
scale = MinMaxScaler()
X_train_transformed = scale.fit_transform(X_train_comb)
X_test_transformed = scale.transform(X_test_comb)

In [135]:
ridge = Ridge(alpha=0.5)
ridge.fit(X_train_transformed, y_train)

lasso = Lasso(alpha=0.5)
lasso.fit(X_train_transformed, y_train)

lin = LinearRegression()
lin.fit(X_train_transformed, y_train)

LinearRegression()

In [136]:
# Generate preditions for training and test sets
y_h_ridge_train = ridge.predict(X_train_transformed)
y_h_ridge_test = ridge.predict(X_test_transformed)

# y_h_lasso_train = np.reshape(lasso.predict(X_train_transformed), (274, 1))
# y_h_lasso_test = np.reshape(lasso.predict(X_test_transformed), (118, 1))

y_h_lin_train = lin.predict(X_train_transformed)
y_h_lin_test = lin.predict(X_test_transformed)

In [137]:
print('Train Error Ridge Model', np.sum((y_train - y_h_ridge_train)**2))
print('Test Error Ridge Model', np.sum((y_test - y_h_ridge_test)**2))
print('\n')

# print('Train Error Lasso Model', np.sum((y_train - y_h_lasso_train)**2))
# print('Test Error Lasso Model', np.sum((y_test - y_h_lasso_test)**2))
# print('\n')

print('Train Error Unpenalized Linear Model', np.sum((y_train - lin.predict(X_train_transformed))**2))
print('Test Error Unpenalized Linear Model', np.sum((y_test - lin.predict(X_test_transformed))**2))

Train Error Ridge Model 26383661408.565697
Test Error Ridge Model 8326044669.98667


Train Error Unpenalized Linear Model 25932742142.0
Test Error Unpenalized Linear Model 2.3748952309564082e+33


In [138]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Impute missing values with median using SimpleImputer
impute = SimpleImputer(strategy='median')
X_train_imputed = impute.fit_transform(X_train_comb)
X_test_imputed = impute.transform(X_test_comb)

# Fit the model
linreg = LinearRegression()
linreg.fit(X_train_comb, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train_comb, y_train))
print('Test r^2:', linreg.score(X_test_imputed, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_imputed)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_imputed)))

Training r^2: 0.7124668119936662
Test r^2: -2969817011576.7217
Training MSE: 6854755.7294516675
Test MSE: 6.35580993354705e+19


In [139]:
from sklearn.preprocessing import StandardScaler

# Scale the train and test data
ss = StandardScaler()
X_train_imputed_scaled = ss.fit_transform(X_train_imputed)
X_test_imputed_scaled = ss.transform(X_test_imputed)

# Fit the model 
linreg_norm = LinearRegression()
linreg_norm.fit(X_train_imputed_scaled, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg_norm.score(X_train_imputed_scaled, y_train))
print('Test r^2:', linreg_norm.score(X_test_imputed_scaled, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_norm.predict(X_train_imputed_scaled)))
print('Test MSE:', mean_squared_error(y_test, linreg_norm.predict(X_test_imputed_scaled)))

Training r^2: 0.712116285563427
Test r^2: -3.0580662956169115e+24
Training MSE: 6863112.236304552
Test MSE: 6.544675332976734e+31


In [140]:
mean_squared_error(y_test, linreg_norm.predict(X_test_imputed_scaled))

6.544675332976734e+31

In [141]:
np.sqrt(mean_squared_error(y_test, linreg_norm.predict(X_test_imputed_scaled)))

8089916769025954.0